In [1]:
import sys,os
sys.path.append("../")

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from IPython.display import HTML
from sklearn.metrics import pairwise_distances  


%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy import stats
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import entropy
import itertools

import seaborn as sns


from sklearn.metrics import adjusted_mutual_info_score
from sklearn.metrics import pairwise_distances  
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import mutual_info_score
from sklearn.metrics import normalized_mutual_info_score

### Function to take weighted entropy

In [2]:
def weighted_entropy(dataframe, entropy_column, group_columns="cluster", ):
    group_size = dataframe.groupby(group_columns).size()
    group_entropy = dataframe.groupby(group_columns)[entropy_column].apply(lambda x: entropy(x.value_counts().values))
    weighted_entropy = (group_size * group_entropy).mean()
    return weighted_entropy

## reading the change object.

In [3]:
article_name = "John_Logie_Baird"
change_object_dir =  "../data/change objects/"
content_dir = "../data/content/"
filename =  f"{article_name}_change.h5"
change_object_file = os.path.join(change_object_dir, filename)
filename = article_name + ".h5"
filepath = os.path.join(content_dir, filename)
with pd.HDFStore(filepath, 'r') as store:
    token_string_df = store.get("all_tokens")

token_string_df = token_string_df.set_index("token_id")["str"]
token_string_df[-1] = "St@rt"
token_string_df[-2] = "$nd"


if os.path.exists(change_object_file):
    with pd.HDFStore(change_object_file, 'r') as store:
        change_object_dataframe = store.get("data")
else:
    print("file do not exist")
change_object_dataframe.shape


change_object_dataframe["ins_length"] = change_object_dataframe["ins_tokens"].str.len()
change_object_dataframe["del_length"] = change_object_dataframe["del_tokens"].str.len()

change_object_dataframe["del_string_tokens"] = change_object_dataframe["del_tokens"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist()))

change_object_dataframe["ins_string_tokens"] = change_object_dataframe["ins_tokens"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist()))
change_object_dataframe["edit_string_tokens"] = change_object_dataframe["ins_string_tokens"] + change_object_dataframe["del_string_tokens"]



change_object_dataframe["left_context"] = change_object_dataframe["left_token"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist())).str.join(" ")


change_object_dataframe["right_context"] = change_object_dataframe["right_token"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist())).str.join(" ")

change_object_dataframe["bykau_cluster"] = pd.Series(-99,index=change_object_dataframe.index)

# change_object_dataframe = change_object_dataframe[["left_context","del_string_tokens","ins_string_tokens", "right_context", 
#                                                    "ins_length", "del_length", "bykau_cluster" ]]
change_object_dataframe.head()

ins_start_pos  \
from revision id to revision id timestamp           timegap           editor                    
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           10.0   
                                                                             1           32.0   
                                                                             2           34.0   
                                                                             3           50.0   
                                                                             4           57.0   

                                                                                ins_end_pos  \
from revision id to revision id timestamp           timegap           editor                  
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0         30.0   
                                                                             1         32.0   
                                                                             2         34.0   
                                                                             3         55.0   
                                                                             4         73.0   

                                                                                left_neigh  \
from revision id to revision id timestamp           timegap           editor                 
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           9   
                                                                             1          10   
                                                                             2          11   
                                                                             3          26   
                                                                             4          32   

                                                                                right_neigh  \
from revision id to revision id timestamp           timegap           editor                  
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           10   
                                                                             1           11   
                                                                             2           12   
                                                                             3           32   
                                                                             4           36   

                                                                                del_start_pos  \
from revision id to revision id timestamp           timegap           editor                    
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           -1.0   
                                                                             1           -1.0   
                                                                             2           -1.0   
                                                                             3           27.0   
                                                                             4           33.0   

                                                                                del_end_pos  \
from revision id to revision id timestamp           timegap           editor                  
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0         -1.0   
                                                                             1         -1.0   
                                                                             2         -1.0   
                                                                             3         31.0   
                                                                             4         35.0   

                                                               

## Reduce the change object.

In [ ]:
ins_and_del = change_object_dataframe[(change_object_dataframe["ins_string_tokens"]!=()) & (change_object_dataframe["del_string_tokens"]!=())]
display(ins_and_del.shape)

### Removing change object with insert or delete token size more than five.

In [ ]:
reduced_ins_and_del = ins_and_del[~((ins_and_del["ins_length"] >5 ) | (ins_and_del["del_length"] >5) )]
reduced_ins_and_del.shape

In [ ]:
reduced_ins_and_del.reset_index(drop=True)[["ins_string_tokens", "del_string_tokens"]].sample(3)

### Removing low user support tokens

In [ ]:
bykau_change_object = reduced_ins_and_del.groupby("ins_string_tokens").filter(lambda x : x.index.get_level_values("editor").nunique()>=2)
bykau_change_object = bykau_change_object.groupby("del_string_tokens").filter(lambda x : x.index.get_level_values("editor").nunique()>=2)
bykau_change_object.shape

#### read annotations

In [4]:
file_name = article_name + "_FULL.csv"
annotation_dir = "../data/annotation/"
full_file_path = os.path.join(annotation_dir, file_name)
annotation_df = pd.read_csv(full_file_path)
annotation_df = annotation_df[["revid_ctxt", "token_id",
                               "rev_id", "nationality", "birth_place","Bulk" ]]
true_labels = np.zeros((annotation_df.shape[0]))
true_labels[(annotation_df["nationality"].str.strip() == "Y").values] = 1
annotation_df["nationality"] = true_labels

In [ ]:
# file_name = article_name + "_FULL.csv"
# annotation_dir = "../data/annotation/"
# full_file_path = os.path.join(annotation_dir, file_name)
# annotation_df = pd.read_csv(full_file_path)

## Function for weighted entropy

In [5]:
def weighted_token_entropy(dataframe, group_by):
    cluster_sizes = dataframe.groupby(group_by).size()
    token_entropy_clusters = dataframe.groupby(group_by)["edit_string_tokens"].apply(
                    lambda token_tuples: entropy(pd.Series(
                    [token for token_tuple in token_tuples.tolist() for token in token_tuple]
                    ).value_counts().values))
    cluster_entropy = (cluster_sizes * token_entropy_clusters).sum()
    return cluster_entropy

### Clustering using jaccard

##### Defining jaccard similarity function.

In [6]:
def bykau_distances(dataframe, context = 8):
    left_neighbours = dataframe["left_context"].apply(lambda x: x.split(" ")[-context:])
    right_neighbours = dataframe["right_context"].apply(lambda x: x.split(" ")[:context])
    
    neighbour_tokens = left_neighbours + right_neighbours
    neighbour_tokens_set = neighbour_tokens.apply(lambda x: np.unique(x))
    
    neighbour_vec = MultiLabelBinarizer().fit_transform(neighbour_tokens_set)
    return pairwise_distances(neighbour_vec, metric="jaccard")
#     db = DBSCAN(eps=eps, min_samples=min_samples, metric='jaccard').fit(neighbour_vec)
#     return db.labels_ 

In [7]:
def bykau_cluster(distances, eps=0.75, min_samples=5):
    db = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed').fit(distances)
    return db.labels_ 


In [ ]:
# %%time
# r_threshold = 8
# cutoff_threshold = 0.75
# # edit_tokens = change_object_dataframe["ins_tokens"] + change_object_dataframe["del_tokens"]
# left_neighbours = change_object_dataframe["left_context"].apply(lambda x: x.split(" ")[-r_threshold:])
# right_neighbours = change_object_dataframe["right_context"].apply(lambda x: x.split(" ")[:r_threshold])
# neighbour_tokens = left_neighbours + right_neighbours

# # bykau_change_object["edit_tokens"] = edit_tokens.apply(lambda x: np.unique(x))
# neighbour_tokens_set = neighbour_tokens.apply(lambda x: np.unique(x))
# neighbour_vec = MultiLabelBinarizer().fit_transform(neighbour_tokens_set)

# db = DBSCAN(eps=cutoff_threshold, min_samples=5, metric='jaccard').fit(neighbour_vec)
# # change_object_dataframe.loc[change_object_dataframe.index,"bykau_cluster"] = db.labels_

#### Saving change object and its clusters

In [ ]:
# bykau_dir =  "../data/bykau_change_object/"
# filename =  f"{article_name}_without_optimization.h5"

# change_object_file = os.path.join(bykau_dir, filename)
# with pd.HDFStore(change_object_file, 'w') as store:
#     store.put("data", change_object_dataframe["bykau_cluster"],)

### Uncomment for seeing the content of each cluster in an ipywidget User interface

In [ ]:
# repers_first =  change_object_dataframe.groupby("bykau_cluster")[["left_context","del_string_tokens", "ins_string_tokens", "right_context"]].apply(lambda x: x.style.render())
# @interact( clusters_html=fixed(repers_first), group=change_object_dataframe["bykau_cluster"].unique().tolist())
# def display_clusters(clusters_html, group):
#      return display(HTML(clusters_html.loc[group]))

#### Making annotation file with cluster

In [ ]:
# def token_in_gap(ann, gap):
#     context_gap = gap.loc[ann[['revid_ctxt', 'rev_id']]]
#     clusters = context_gap.loc[ context_gap["token_id"].apply(
#             lambda x: ann["token_id"] in x), ["bykau_cluster"]].values
#     if clusters.size >0:
#             clusters = pd.Series(clusters[0],index=["bykau_cluster",])
#     else:
#         clusters = pd.Series([-10], index=["bykau_cluster",])
#     return clusters

In [8]:
def token_in_gap(ann, gap_df, gap_cluster_df):
    context_gap = gap_df.loc[ann[['revid_ctxt', 'rev_id']]]
    context_cluster = gap_cluster_df.loc[ann[['revid_ctxt', 'rev_id']]]
    clusters = context_cluster.loc[ context_gap["token_id"].apply(
            lambda x: ann["token_id"] in x),:].values
    if clusters.size >0:
            clusters = pd.Series(clusters[0],index=gap_cluster_df.columns)
    else:
        clusters = pd.Series(-10, index=gap_cluster_df.columns)
    return clusters

In [9]:
def evaluate_bykau(change_object_dataframe, annotation_df):
    ins_array = change_object_dataframe.reset_index().loc[
    change_object_dataframe["ins_start_pos"].values != -1, 
                  ["to revision id","ins_tokens", 'to revision id', "bykau_cluster"]].values

    # delete array is always done in from revision so taking it and leaving other change object where delete does not come.
    del_array = change_object_dataframe.reset_index().loc[
    change_object_dataframe["del_start_pos"].values != -1, 
                  ["from revision id","del_tokens", 'to revision id',"bykau_cluster"]].values

    gap_array = np.concatenate([ins_array,del_array], axis=0)
    gap_df = pd.DataFrame(gap_array,columns= ["revid_ctxt", "token_id", "rev_id","bykau_cluster"])
    gap_df = gap_df.set_index(['revid_ctxt', 'rev_id'])
    
    annotation_df["bykau_cluster"] = annotation_df.apply(token_in_gap, axis=1, args=(gap_df,))
    nationality_cluster = np.zeros((annotation_df.shape[0]))
    nationality_cluster[annotation_df["nationality"].str.strip() == "Y"] = 1
    annotation_df["nationality_cluster"] = nationality_cluster

    evaluation_score = pd.Series(index=["rand", "entropy",])
    evaluation_score["rand"] = adjusted_rand_score( annotation_df["bykau_cluster"], nationality_cluster)
    evaluation_score["entropy"] = weighted_entropy(annotation_df, group_columns="bykau_cluster", entropy_column="nationality_cluster")
    return evaluation_score   

In [ ]:
context_array  = np.array([2, 4, 8, 15, 30])
eps_array =[0.001, 0.025, 0.05, 0.1, 0.2, 0.4, 0.8]

min_samples_array = 2 
all_combinations = list(itertools.product(context_array, eps_array))

idx = pd.MultiIndex.from_product([context_array, eps_array],
                                names=["context","eps"])


all_combinations = list(itertools.product(context_array, eps_array))


bykau_cluster_df =  pd.DataFrame(columns=idx, index= bykau_change_object.index) #pd.DataFrame(columns=idx)

bykau_evaluation_df = pd.DataFrame(index=idx, columns=["rand", "entropy"])


In [ ]:
for context in context_array:
    distances = bykau_distances(bykau_change_object, context = context)
    for eps in eps_array:
        clusters = bykau_cluster(distances, eps=eps, min_samples=2)
        
        bykau_cluster_df.loc[bykau_change_object.index, (context,eps)] = pd.Series(clusters, index=bykau_change_object.index)

In [ ]:
# insert array is always done in to revision so taking it and leaving other change object where 
ins_array = bykau_change_object.reset_index().loc[
    bykau_change_object["ins_start_pos"].values != -1, 
                  ["to revision id","ins_tokens", 'to revision id']].values
ins_cluster = bykau_cluster_df.loc[
    bykau_change_object["ins_start_pos"].values != -1, :]

# delete array is always done in from revision so taking it and leaving other change object where delete does not come.
del_array = bykau_change_object.reset_index().loc[
    bykau_change_object["del_start_pos"].values != -1, 
                  ["from revision id","del_tokens", 'to revision id']].values
del_cluster = bykau_cluster_df.loc[
    bykau_change_object["del_start_pos"].values != -1, :]

gap_array = np.concatenate([ins_array,del_array], axis=0)
gap_df = pd.DataFrame(gap_array,columns=["revid_ctxt", "token_id",
                               "rev_id"])

gap_cluster= pd.concat([ins_cluster, del_cluster], axis=0)
gap_df = gap_df.set_index(['revid_ctxt', 'rev_id'])
gap_cluster_df = pd.concat([ins_cluster, del_cluster], axis=0)

gap_cluster_df.index=gap_df.index

In [ ]:
# Finding the tokens who were in the gap.
al_combination_clusters_df = annotation_df.apply(token_in_gap, axis=1, args=(gap_df, gap_cluster_df))

annotation_clusters = pd.concat([annotation_df, al_combination_clusters_df], axis=1)

annotation_clusters.head()
#true_labels[true_lable_df["birth_place"].str.strip() == "Y"] = 2

In [ ]:
a = annotation_df.loc[al_combination_clusters_df.mean(axis=1) > -10]
b = al_combination_clusters_df.loc[al_combination_clusters_df.mean(axis=1) > -10]

annotation_clusters = pd.concat([a,b], axis=1)

In [ ]:
df2 = gap_df.copy()
df2=df2.reset_index()
df2['nationality'] = 0

aci = annotation_clusters.set_index(['revid_ctxt', 'rev_id']).sort_index()
aci = aci[aci['Bulk'] =='N']
aci_y=aci[aci['nationality'] == 1]
aci_n=aci[aci['nationality'] == 0]

counter = 0
def nat_val(row):
    global counter
    val = 0
    x=0
    y=0
    try:
        x = int(aci_y.loc[(row[0],row[1]), ['token_id']].isin(row[2]).sum())
        val = val + (1 if x > 0 else 0)
        
    except KeyError as e:
        pass
    try:
        y = int(aci_n.loc[(row[0],row[1]), ['token_id']].isin(row[2]).sum())
        val = val - (1 if y > 0 else 0)
        counter += y
    except KeyError as e:
        pass
#     if (1 if x > 0 else 0) + (1 if y > 0 else 0) > 1:
#         try:
#             if not (aci.loc[(row[0],row[1]),['Bulk']] == 'Y').all()[0]:
#                 print((aci.loc[(row[0],row[1]),['token_id','nationality','Bulk']])[aci.loc[(row[0],row[1]), 'token_id'].isin(row[2])])
#                 print(aci.loc[(row[0],row[1]), ['token_id']].isin(row[2]))
#         except:
#             import pdb; pdb.set_trace()
    return val 
    
df2['nationality'] = df2[['revid_ctxt', 'rev_id', 'token_id']].apply(nat_val, axis=1)
df3 = pd.concat([df2, gap_cluster_df.reset_index()], axis=1)

In [ ]:
df3['nationality'].value_counts()

In [ ]:
entropies = []
print("Without bulks")
for context, eps in all_combinations:
    print(str((context, eps)) + ": " + str(weighted_entropy(df3, entropy_column="nationality", group_columns=(context, eps))))

In [ ]:
df4 = annotation_clusters[annotation_clusters['Bulk']=='N']
for context, eps in all_combinations:
    bykau_evaluation_df.loc[(context, eps),"entropy"] = weighted_entropy(df4, 
                                                                                entropy_column="nationality", 
                                                                                group_columns=(context, eps))
bykau_evaluation_df["entropy"].sort_values().iloc[0:50]

In [ ]:
annotation_clusters['Bulk'].value_counts()

In [ ]:
for context, eps in all_combinations:
    bykau_evaluation_df.loc[(context, eps),"entropy"] = weighted_entropy(annotation_clusters, 
                                                                                entropy_column="nationality", 
                                                                                group_columns=(context, eps))
#     bykau_evaluation_df.loc[(context, eps),"rand"] = adjusted_rand_score(annotation_clusters[(context, 
#                                                                                                      eps)], 
#                                                                                 true_labels)

In [ ]:
bykau_evaluation_df["entropy"].sort_values().iloc[0:20]

In [10]:
context_array  = np.array([2, 4, 8, 15, 30])
eps_array =[0.001, 0.025, 0.05, 0.1, 0.2, 0.4, 0.8]

min_samples_array = [2] 
# all_combinations = list(itertools.product(context_array, eps_array))
all_combinations_without_optimization = list(itertools.product(context_array, eps_array))

idx_without_optimization  = pd.MultiIndex.from_product([context_array, eps_array],
                                names=["context","eps"])

# idx_without_optimization = pd.MultiIndex.from_product([context_array, eps_array, min_samples_array],
#                                 names=["context","eps", "min_samples"])
bykau_evaluation_without_optimization = pd.DataFrame(index=idx_without_optimization, 
                                                     columns=["rand", "entropy"])



bykau_without_optimization_cluster_df = pd.DataFrame(columns=idx_without_optimization)



In [11]:
%%time
for context in context_array:
    distances = bykau_distances(change_object_dataframe, context = context)
    for eps in eps_array:
        clusters = bykau_cluster(distances, eps=eps, min_samples=2)
        
        bykau_without_optimization_cluster_df[context,eps] = pd.Series(clusters)

/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec

CPU times: user 1min 55s, sys: 12.6 s, total: 2min 7s
Wall time: 1min 48s


In [12]:
# insert array is always done in to revision so taking it and leaving other change object where 
ins_array = change_object_dataframe.reset_index().loc[
    change_object_dataframe["ins_start_pos"].values != -1, 
                  ["to revision id","ins_tokens", 'to revision id']].values
ins_cluster = bykau_without_optimization_cluster_df.loc[
    change_object_dataframe["ins_start_pos"].values != -1, :]

# delete array is always done in from revision so taking it and leaving other change object where delete does not come.
del_array = change_object_dataframe.reset_index().loc[
    change_object_dataframe["del_start_pos"].values != -1, 
                  ["from revision id","del_tokens", 'to revision id']].values
del_cluster = bykau_without_optimization_cluster_df.loc[
    change_object_dataframe["del_start_pos"].values != -1, :]

gap_array = np.concatenate([ins_array,del_array], axis=0)
gap_df = pd.DataFrame(gap_array,columns=["revid_ctxt", "token_id",
                               "rev_id"])

gap_cluster= pd.concat([ins_cluster, del_cluster], axis=0)
gap_df = gap_df.set_index(['revid_ctxt', 'rev_id'])
gap_cluster_df = pd.concat([ins_cluster, del_cluster], axis=0)

gap_cluster_df.index=gap_df.index

# Finding the tokens who were in the gap.
al_combination_clusters_df = annotation_df.apply(token_in_gap, axis=1, args=(gap_df, gap_cluster_df))

annotation_clusters = pd.concat([annotation_df, al_combination_clusters_df], axis=1)



# true_labels = np.zeros((annotation_df.shape[0]))
# true_labels[(annotation_df["nationality"].str.strip() == "Y").values] = 1
# annotation_df["nationality"] = true_labels
#true_labels[true_lable_df["birth_place"].str.strip() == "Y"] = 2



# Calculating the entropy of the Annotated change objects

In [21]:
df2 = gap_df.copy()
df2=df2.reset_index()
df2['nationality'] = 0

aci = annotation_clusters.set_index(['revid_ctxt', 'rev_id']).sort_index()
aci = aci[aci['Bulk'] =='N']
aci_y=aci[aci['nationality'] == 1]
aci_n=aci[aci['nationality'] == 0]

counter = 0
def nat_val(row):
    global counter
    val = 0
    x=0
    y=0
    try:
        x = int(aci_y.loc[(row[0],row[1]), ['token_id']].isin(row[2]).sum())
        val = val + (1 if x > 0 else 0)
        
    except KeyError as e:
        pass
    try:
        y = int(aci_n.loc[(row[0],row[1]), ['token_id']].isin(row[2]).sum())
        val = val - (1 if y > 0 else 0)
        counter += y
    except KeyError as e:
        pass
#     if (1 if x > 0 else 0) + (1 if y > 0 else 0) > 1:
#         try:
#             if not (aci.loc[(row[0],row[1]),['Bulk']] == 'Y').all()[0]:
#                 print((aci.loc[(row[0],row[1]),['token_id','nationality','Bulk']])[aci.loc[(row[0],row[1]), 'token_id'].isin(row[2])])
#                 print(aci.loc[(row[0],row[1]), ['token_id']].isin(row[2]))
#         except:
#             import pdb; pdb.set_trace()
    return val 
    
df2['nationality'] = df2[['revid_ctxt', 'rev_id', 'token_id']].apply(nat_val, axis=1)
df3 = pd.concat([df2, gap_cluster_df.reset_index()], axis=1)

In [15]:
entropies = []
print("Without bulks")
for context, eps in all_combinations_without_optimization:
    print(str((context, eps)) + ": " + str(weighted_entropy(df3, entropy_column="nationality", group_columns=(context, eps))))

Without bulks
(2, 0.001): 0.9687533115695315
(2, 0.025): 0.9687533115695315
(2, 0.05): 0.9687533115695315
(2, 0.1): 0.9687533115695315
(2, 0.2): 0.9687533115695315
(2, 0.4): 3.9587710425096807
(2, 0.8): 156.75197195239275
(4, 0.001): 1.220476592052727
(4, 0.025): 1.220476592052727
(4, 0.05): 1.220476592052727
(4, 0.1): 1.220476592052727
(4, 0.2): 1.1617986579589623
(4, 0.4): 2.532822003972604
(4, 0.8): 445.4838719899523
(8, 0.001): 1.4693867125070161
(8, 0.025): 1.4693867125070161
(8, 0.05): 1.4693867125070161
(8, 0.1): 1.2946428525961646
(8, 0.2): 0.9752636186278807
(8, 0.4): 2.7238185125280636
(8, 0.8): 534.6634030369431
(15, 0.001): 1.5796493907206288
(15, 0.025): 1.5796493907206288
(15, 0.05): 1.414065576153409
(15, 0.1): 1.05857028142598
(15, 0.2): 1.092392932008907
(15, 0.4): 4.296414081718749
(15, 0.8): 668.5617760943499
(30, 0.001): 1.6106862502595019
(30, 0.025): 1.433799901850248
(30, 0.05): 1.13415447609199
(30, 0.1): 1.0388522931176771
(30, 0.2): 1.7923024966010208
(30, 0.4

# Calculating the entropy of the Annotated cases

In [19]:
a = annotation_df.loc[al_combination_clusters_df.mean(axis=1) > -10]
b = al_combination_clusters_df.loc[al_combination_clusters_df.mean(axis=1) > -10]

annotation_clusters = pd.concat([a,b], axis=1)

In [27]:
df4 = annotation_clusters[annotation_clusters['Bulk']=='N']
for context, eps in all_combinations_without_optimization:
    bykau_evaluation_without_optimization.loc[(context, eps),"entropy"] = weighted_entropy(df4, 
                                                                                entropy_column="nationality", 
                                                                                group_columns=(context, eps))
    bykau_evaluation_without_optimization.loc[(context, eps),"rand"] = adjusted_rand_score(df4[(context, 
                                                                                                     eps)], 
                                                                                df4['nationality'])

bykau_evaluation_without_optimization["entropy"].sort_values().iloc[0:25]

context  eps  
2        0.001     1.1264
         0.025     1.1264
         0.050     1.1264
         0.100     1.1264
         0.200     1.1264
30       0.100    1.13915
15       0.100    1.22499
         0.200     1.3654
4        0.100    1.36815
         0.025    1.36815
         0.001    1.36815
         0.050    1.36815
30       0.050    1.41643
4        0.200    1.49722
8        0.200    1.49789
30       0.025    1.69439
         0.001    1.71708
15       0.050    1.81844
8        0.100    1.82507
         0.050    1.91669
         0.025    1.91669
         0.001    1.91669
30       0.200    1.93159
15       0.001     1.9481
         0.025     1.9481
Name: entropy, dtype: object

### **@Aadil**, the rankindex here is wrong because all the nationality clusters hould be mark as one

In [28]:
bykau_evaluation_without_optimization

rand  entropy
context eps                       
2       0.001   0.0344953   1.1264
        0.025   0.0344953   1.1264
        0.050   0.0344953   1.1264
        0.100   0.0344953   1.1264
        0.200   0.0344953   1.1264
        0.400   0.0660082  15.8781
        0.800           0  509.252
4       0.001   0.0233593  1.36815
        0.025   0.0233593  1.36815
        0.050   0.0233593  1.36815
        0.100   0.0233593  1.36815
        0.200   0.0293159  1.49722
        0.400    0.127709  7.89078
        0.800           0  509.252
8       0.001   0.0155554  1.91669
        0.025   0.0155554  1.91669
        0.050   0.0155554  1.91669
        0.100   0.0168361  1.82507
        0.200   0.0448379  1.49789
        0.400   0.0843053  6.29846
        0.800           0  509.252
15      0.001  0.00679509   1.9481
        0.025  0.00679509   1.9481
        0.050   0.0130613  1.81844
        0.100   0.0343277  1.22499
        0.200   0.0627298   1.3654
        0.400   0.0199065  9.90127
        0.800           0  509.252
30      0.001  0.00629528  1.71708
        0.025   0.0102674  1.69439
        0.050   0.0289889  1.41643
        0.100   0.0530047  1.13915
        0.200   0.0592763  1.93159
        0.400   0.0214034  17.5753
        0.800           0  509.252

In [ ]:
# for context, eps, min_samples in all_combinations_without_optimization:
#     print(f"processing eps {eps}, min samples {min_samples} and context {context}")
#     change_object_dataframe.loc[change_object_dataframe.index,
#                             "bykau_cluster"] = bykau_cluster(change_object_dataframe, 
#                                                              context=int(context), eps=eps, min_samples=min_samples)
#     bykau_evaluation_without_optimization.loc[context, eps, 
#                             min_samples] =evaluate_bykau(change_object_dataframe, annotation_df)


In [ ]:
#bykau_evaluation_without_optimization["entropy"].sort_values().iloc[0:200]

bykau_evaluation_without_optimization.reset_index().set_index(["eps", "context"])["entropy"].sort_values().iloc[0:50]

In [ ]:
# # insert array is always done in to revision so taking it and leaving other change object where 
# ins_array = change_object_dataframe.reset_index().loc[
#     change_object_dataframe["ins_start_pos"].values != -1, 
#                   ["to revision id","ins_tokens", 'to revision id']].values

# # delete array is always done in from revision so taking it and leaving other change object where delete does not come.
# del_array = change_object_dataframe.reset_index().loc[
#     change_object_dataframe["del_start_pos"].values != -1, 
#                   ["from revision id","del_tokens", 'to revision id',"bykau_cluster"]].values

# gap_array = np.concatenate([ins_array,del_array], axis=0)
# gap_df = pd.DataFrame(gap_array,columns= ["revid_ctxt", "token_id", "rev_id","bykau_cluster"])
# gap_df = gap_df.set_index(['revid_ctxt', 'rev_id'])

# Finding the tokens who were in the gap.
# annotation_df["bykau_cluster"] = annotation_df.apply(token_in_gap, axis=1, args=(gap_df,))

# nationality_cluster = np.zeros((annotation_df.shape[0]))
# nationality_cluster[annotation_df["nationality"].str.strip() == "Y"] = 1
# annotation_df["nationality_cluster"] = nationality_cluster

# evaluation_score = pd.Series(index=["rand", "entropy",])
# evaluation_score["rand"] = adjusted_rand_score( annotation_df["bykau_cluster"], nationality_cluster)
# evaluation_score["entropy"] = weighted_entropy(annotation_df, group_columns="bykau_cluster", entropy_column="nationality_cluster")
# evaluation_score

# both_cluster = np.zeros((annotation_df.shape[0]))
# both_cluster[annotation_df["nationality"].str.strip() == "Y"] = 1
# both_cluster[annotation_df["birth_place"].str.strip() == "Y"] = 2
# annotation_df["both_cluster"] = both_cluster

# evaluation_score = pd.Series(index=["rand", "entropy", ])
# evaluation_score["rand"] = adjusted_rand_score( annotation_df["bykau_cluster"], both_cluster)

# evaluation_score["entropy"] = weighted_entropy(annotation_df, group_columns="bykau_cluster", entropy_column="both_cluster")

# evaluation_score


In [ ]:
def weighted_token_entropy(dataframe, group_by):
    cluster_sizes = dataframe.groupby(group_by).size()
    token_entropy_clusters = dataframe.groupby(group_by)["edit_string_tokens"].apply(
                    lambda token_tuples: entropy(pd.Series(
                    [token for token_tuple in token_tuples.tolist() for token in token_tuple]
                    ).value_counts().values))
    cluster_entropy = (cluster_sizes * token_entropy_clusters).sum()
    return cluster_entropy

In [ ]:
bykau_cluster_df.index = change_object_dataframe.index

bykau_optimised = pd.concat([change_object_dataframe, bykau_cluster_df], axis=1)

In [ ]:
entropy_series_with_optimization = pd.Series(index=all_combinations)

for context_eps_tuple in all_combinations_without_optimization:
    bykau_evaluation_df.loc[context_eps_tuple,
                                              "token_entropy"] = weighted_token_entropy(bykau_optimised, 
                                                                                        context_eps_tuple)
#     entropy_series_with_optimization[context_eps_tuple]= weighted_token_entropy(bykau_optimised, context_eps_tuple)
# all_combinations_without_optimization[0]

In [ ]:
# entropy_series_with_optimization.sort_values()

In [ ]:
bykau_without_optimization_cluster_df.index = change_object_dataframe.index

without_optimised = pd.concat([change_object_dataframe, bykau_without_optimization_cluster_df], axis=1)

In [ ]:
# entropy_series_without_optimization = pd.Series(index=all_combinations_without_optimization)

for context_eps_tuple in all_combinations_without_optimization:
    bykau_evaluation_without_optimization.loc[context_eps_tuple,
                                              "token_entropy"] = weighted_token_entropy(without_optimised, 
                                                                                        context_eps_tuple)

#     entropy_series_without_optimization[context_eps_tuple]= weighted_token_entropy(without_optimised, context_eps_tuple)
# all_combinations_without_optimization[0]

In [ ]:
# entropy_series_without_optimization.sort_values()

In [ ]:
bykau_evaluation_without_optimization.head()

In [ ]:
bykau_evaluation_df.head()

In [ ]:
bykau_evaluation_df["token_entropy"].sort_values().iloc[0:20]

In [ ]:
bykau_evaluation_without_optimization["token_entropy"].sort_values().iloc[-20:]

In [ ]:
bykau_evaluation_without_optimization["entropy"].sort_values().iloc[0:20]

In [ ]:
bykau_evaluation_df["entropy"].sort_values().iloc[0:20]

In [ ]:
bykau_evaluation_df.reset_index().set_index(["eps", "context"])["entropy"].sort_values().iloc[0:50]

In [ ]:
bykau_evaluation_df.astype(np.float64).corr()

In [ ]:
bykau_evaluation_without_optimization.astype(np.float64).corr()